In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import datetime
#from keras.preprocessing.sequence import pad_sequences
#from keras.layers import Embedding, Input, LSTM
#from keras.models import Sequential, Model,load_model
#from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input, Convolution1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import datetime

In [2]:
data=pd.read_csv("train.csv")

C:\Users\Pranav\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
max(data['unit_sales'])

89440.0

In [4]:
min(data['unit_sales'])

-15372.0

In [5]:
data.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [6]:
data['store_nbr'].unique()

array([25,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 23, 24, 26, 27, 28, 30, 31, 32, 33, 34, 35, 37, 38, 39,
       40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 36, 53, 20, 29, 21,
       42, 22, 52], dtype=int64)

In [7]:
store_data=pd.read_csv("stores.csv")
items_data=pd.read_csv("items.csv")
transactions_data=pd.read_csv("transactions.csv")
holiday_data=pd.read_csv("holidays_events.csv")

In [8]:
reg_dict = {'El Oro': 'Coast','Esmeraldas': 'Coast','Santo Domingo de los Tsachilas':'Coast','Guayas':'Coast','Los Rios':'Coast'
           ,'Manabi':'Coast','Santa Elena':'Coast',
           'Azuay':'Sierra','Bolivar':'Sierra','Cotopaxi':'Sierra','Chimborazo':'Sierra','Imbabura':'Sierra'
            ,'Loja':'Sierra','Pichincha':'Sierra','Tungurahua':'Sierra','Pastaza':'Amazon'}

In [9]:
store_data['Geographic Region'] = store_data['state'].map(reg_dict)

In [10]:
store_data.drop(["city","state","type","cluster"],inplace=True,axis=1)

In [11]:
items_data.drop("class",axis=1,inplace=True)

### Stores

In [12]:
merged_data=data.merge(store_data,how="left",on="store_nbr")

In [13]:
agg_data=pd.DataFrame({'unit_sales' : merged_data.groupby(["store_nbr","date"])["unit_sales"].sum(),
                       'region':merged_data.groupby(["store_nbr","date"])["Geographic Region"].max()}).reset_index()

In [14]:
agg_data=agg_data.merge(transactions_data,how="left",on=["date","store_nbr"])

In [15]:
import pickle
scaler = MinMaxScaler()
scaler.fit(np.array(agg_data.unit_sales).reshape(-1,1))
scalerfile = 'store_scaler.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

In [16]:
scaler = MinMaxScaler()
agg_data["unit_sales"]=scaler.fit_transform(np.array(agg_data.unit_sales).reshape(-1,1))
scaler1 = MinMaxScaler()
agg_data["transactions"]=scaler.fit_transform(np.array(agg_data.transactions).reshape(-1,1))

In [17]:
final_agg_data=pd.DataFrame()
for i in agg_data.store_nbr.unique():
    temp_data=agg_data[agg_data.store_nbr==i].reset_index(drop=True)
    for i in range(1,8):
        temp_data["t"+str(i)]=temp_data["unit_sales"].shift(i)
    temp_data.drop(temp_data[temp_data.isnull().any(axis=1)].index,inplace=True)
    final_agg_data=pd.concat([final_agg_data,temp_data])

In [18]:
final_agg_data.reset_index(inplace=True,drop=True)

In [19]:
final_agg_data["date"]=pd.to_datetime(final_agg_data.date)

In [20]:
final_agg_data["weekday"]=[i.weekday() for i in final_agg_data.date]
final_agg_data["month"]=[i.month for i in final_agg_data.date]

In [21]:
final_agg_data=pd.concat([pd.get_dummies(final_agg_data[['store_nbr', 'region','weekday', 'month']].astype(str)),
          final_agg_data[['date','transactions', 't1', 't2', 't3','t4', 't5', 't6', 't7','unit_sales']]],axis=1)

In [22]:
final_agg_data.to_csv("stores_agg.csv",index=None)

In [39]:
final_agg_data.drop("date",inplace=True,axis=1)

### Items

In [22]:
merged_data_items=data.merge(items_data,how="left",on="item_nbr")

In [23]:
agg_data_items=pd.DataFrame({'unit_sales' : merged_data_items.groupby(["date","family"])["unit_sales"].sum(),
                       'perishable':merged_data_items.groupby(["date","family"])["perishable"].mean()}).reset_index()

In [24]:
#scaler.transform(np.array(agg_data_items.unit_sales).reshape(-1,1))

In [25]:
import pickle
scaler = MinMaxScaler()
scaler.fit(np.array(agg_data_items.unit_sales).reshape(-1,1))
scalerfile = 'item_scaler.sav'
#pickle.dump(scaler, open(scalerfile, 'wb'))

In [33]:
min(agg_data_items['unit_sales'])

-1028.0

In [16]:
min(agg_data['unit_sales'])

-5220.266999999998

In [27]:
scaler.transform(np.array(agg_data_items.unit_sales).reshape(-1,1)).reshape(1,len(np.array(agg_data_items.unit_sales)))[0]

array([0.00176294, 0.00314591, 0.00206861, ..., 0.21589537, 0.00608985,
       0.00342007])

In [28]:
agg_data_items['scaled'] = scaler.transform(np.array(agg_data_items.unit_sales).reshape(-1,1)).reshape(1,len(np.array(agg_data_items.unit_sales)))[0]
agg_data_items.head()

,date,family,unit_sales,perishable,scaled
0,2013-01-01,BEAUTY,2.000,0,0.001763
1,2013-01-01,BEVERAGES,810.000,0,0.003146
2,2013-01-01,BREAD/BAKERY,180.589,1,0.002069
3,2013-01-01,CLEANING,186.000,0,0.002078
4,2013-01-01,DAIRY,143.000,1,0.002004


In [32]:
scaler.inverse_transform(np.array(agg_data_items['scaled'][0]).reshape(-1,1))

array([[2.]])

In [34]:
np.array(agg_data_items.unit_sales)

array([0.00176294, 0.00314591, 0.00206861, ..., 0.21589537, 0.00608985,
       0.00342007])

In [26]:
scaler2 = MinMaxScaler()
agg_data_items["unit_sales"]=scaler2.fit_transform(np.array(agg_data_items.unit_sales).reshape(-1,1))

In [27]:
final_agg_data_items=pd.DataFrame()
for i in agg_data_items.family.unique():
    temp_data=agg_data_items[agg_data_items.family==i].reset_index(drop=True)
    for i in range(1,8):
        temp_data["t"+str(i)]=temp_data["unit_sales"].shift(i)
    temp_data.drop(temp_data[temp_data.isnull().any(axis=1)].index,inplace=True)
    final_agg_data_items=pd.concat([final_agg_data_items,temp_data])

In [28]:
final_agg_data_items.reset_index(inplace=True,drop=True)

In [29]:
final_agg_data_items["date"]=pd.to_datetime(final_agg_data_items.date)

In [30]:
final_agg_data_items["weekday"]=[i.weekday() for i in final_agg_data_items.date]
final_agg_data_items["month"]=[i.month for i in final_agg_data_items.date]

In [31]:
final_agg_data_items=pd.concat([pd.get_dummies(final_agg_data_items[['family','weekday', 'month']].astype(str)),
          final_agg_data_items[['date','t1', 't2', 't3','t4', 't5', 't6', 't7','unit_sales']]],axis=1)

In [31]:
final_agg_data_items.to_csv("items_agg.csv",index=None)

In [14]:
final_agg_data_items.drop("date",inplace=True,axis=1)

### Final Model

In [32]:
agg_data_final=data.merge(store_data,how="left",on="store_nbr")

In [33]:
agg_data_final=agg_data_final.merge(items_data,on="item_nbr",how="left")

In [34]:
temp_unit_sales=agg_data_final.groupby(["date","store_nbr","family"])["unit_sales"].sum()

In [35]:
temp_perishable=agg_data_final.groupby(["date","store_nbr","family"])["perishable"].mean()

In [36]:
agg_data_final1=pd.DataFrame({'unit_sales' : temp_unit_sales,
                       'perishable': temp_perishable}).reset_index()

In [37]:
agg_data_final1=agg_data_final1.merge(store_data,on="store_nbr",how="left")

In [38]:
agg_data_final1["date"]=pd.to_datetime(agg_data_final1.date)

In [39]:
agg_data_final1["weekday"]=[i.weekday() for i in agg_data_final1.date]
agg_data_final1["month"]=[i.month for i in agg_data_final1.date]

In [40]:
agg_data_final1.to_csv("agg_final_data.csv",index=False)